In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import warnings
import gc

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

from imblearn.under_sampling import RandomUnderSampler

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
import lightgbm as lgb

warnings.filterwarnings('ignore')

In [3]:
dataset = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [4]:
# Age 범주형 변경

# 구간 설정
bins = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, float('inf')]

# 각 구간에 대한 라벨 설정
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

# Age 열을 범주형 변수로 변환
dataset['AgeGroup'] = pd.cut(dataset['Age'], bins=bins, labels=labels, right=False)

# Age 열 삭제
dataset = dataset.drop(['Age'], axis = 1)

In [5]:
dataset['AgeGroup'] = dataset['AgeGroup'].astype(int)

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cat=['Gender', 'Vehicle_Age', 'Vehicle_Damage']
for column in cat:
    dataset[column] = label_encoder.fit_transform(dataset[column])

In [7]:
# train 데이터의 수치형 컬럼을 추출해서 train_X_num이라는 객체에 저장합니다.
train_X_num = dataset[['Annual_Premium']]
# scikit-learn 패키지의 RobustScaler 클래스를 불러옵니다.
from sklearn.preprocessing import RobustScaler
# RobustScaler 객체를 생성합니다.
robustScaler = RobustScaler()

# fit_transform()을 사용해서 학습과 스케일링을 한 번에 적용합니다.
X_train_robust = robustScaler.fit_transform(train_X_num)
# robust 스케일링이 완료된 데이터를 데이터프레임 형태로 변환합니다.
train_robust = pd.DataFrame(X_train_robust, 
                            index=train_X_num.index, 
                            columns=train_X_num.columns)

In [8]:
train_robust['Annual_Premium'].head()

0    2.347749
1    1.911034
2    0.438761
3   -2.059687
4    0.008960
Name: Annual_Premium, dtype: float64

In [9]:
dataset = dataset.drop('Annual_Premium', axis=1)

In [10]:
df = pd.concat([dataset, train_robust], axis=1)

# 결과 출력
print(df)

                id  Gender  Driving_License  Region_Code  Previously_Insured  \
0                0       1                1         35.0                   0   
1                1       1                1         28.0                   0   
2                2       0                1         14.0                   1   
3                3       0                1          1.0                   0   
4                4       0                1         15.0                   1   
...            ...     ...              ...          ...                 ...   
11504793  11504793       1                1          6.0                   0   
11504794  11504794       0                1         36.0                   0   
11504795  11504795       0                1         32.0                   1   
11504796  11504796       0                1         28.0                   0   
11504797  11504797       1                1         28.0                   1   

          Vehicle_Age  Vehicle_Damage  

In [11]:
df.head()

,id,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Policy_Sales_Channel,Vintage,Response,AgeGroup,Annual_Premium
0,0,1,1,35.0,0,0,1,124.0,187,0,0,2.347749
1,1,1,1,28.0,0,2,1,26.0,288,1,4,1.911034
2,2,0,1,14.0,1,1,0,152.0,254,0,1,0.438761
3,3,0,1,1.0,0,0,1,156.0,76,0,3,-2.059687
4,4,0,1,15.0,1,0,0,152.0,294,0,3,0.008960


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           int64  
 6   Vehicle_Damage        int64  
 7   Policy_Sales_Channel  float64
 8   Vintage               int64  
 9   Response              int64  
 10  AgeGroup              int64  
 11  Annual_Premium        float64
dtypes: float64(3), int64(9)
memory usage: 1.0 GB


In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
cat_cols = ["Gender", "Driving_License", "Previously_Insured", "Vehicle_Age", "Vehicle_Damage"]
num_cols = ["AgeGroup", "Region_Code", "Annual_Premium", "Policy_Sales_Channel", "Vintage"]
target = "Response"

In [15]:
X = df[cat_cols + num_cols]

y = df[target]

In [ ]:
#X["Vehicle_Age"] = X["Vehicle_Age"].replace({0,1,2})
#X = pd.get_dummies(X, columns=cat_cols)



In [16]:
X.head()

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,AgeGroup,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage
0,1,1,0,0,1,0,35.0,2.347749,124.0,187
1,1,1,0,2,1,4,28.0,1.911034,26.0,288
2,0,1,1,1,0,1,14.0,0.438761,152.0,254
3,0,1,0,0,1,3,1.0,-2.059687,156.0,76
4,0,1,1,0,0,3,15.0,0.008960,152.0,294


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

In [18]:
xgb_clf =xgb_clf = XGBClassifier(
    use_label_encoder=False, 
    eval_metric='auc', 
    n_estimators=1500, 
    learning_rate=0.02, 
    max_depth=6, 
    subsample=0.85, 
    colsample_bytree=0.75, 
    gamma=0.2, 
    min_child_weight=1,
    reg_alpha=0.01,
    reg_lambda=0.8,
    device="gpu"
)


xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=0.2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
xgb_pred = xgb_clf.predict_proba(X_test)[:, 1]

xgb_auc = roc_auc_score(y_test, xgb_pred)
print(f"XGBoost AUC: {xgb_auc}")

XGBoost AUC: 0.877343930015247
